# Preparando dados do Spotify para o Tableau

###  Etapa 1: solicitar dados
> https://towardsdatascience.com/visualizing-spotify-data-with-python-tableau-687f2f528cdd

### Etapa 2: preparar dados de streaming/biblioteca

In [1]:
# Importação das bibliotecas
import pandas as pd
import numpy as np
import requests

### Criando streaming/dataframe da biblioteca

In [ ]:
# leia seus 1+ arquivos StreamingHistory (dependendo de quão extenso é seu histórico de streaming) em quadros de dados pandas
df_stream0 = pd.read_json('StreamingHistory0.json')
df_stream1 = pd.read_json('streamingHistory1.json')

# mesclar dataframes de streaming
df_stream = pd.concat([df_stream0, df_stream1])

# crie um 'UniqueID' para cada música combinando os campos 'artistName' e 'trackName'
df_stream['UniqueID'] = df_stream['artistName'] + ":" + df_stream['trackName']

df_stream.head()

### Em seguida, limpei meu arquivo 'YourLibrary' do Spotify para que ele contivesse apenas o dicionário de “faixas”, entre colchetes [ ] e salvei como um novo arquivo 'YourLibrary1'. Alguém melhor em limpar arquivos json provavelmente poderia automatizar esta etapa e usar o arquivo original.

In [ ]:
# leia seu arquivo json da biblioteca editado em um dataframe do pandas
df_library = pd.read_json('YourLibrary1.json')

# adicionar coluna UniqueID (o mesmo que acima)
df_library['UniqueID'] = df_library['artist'] + ":" + df_library['track']

# adicione a coluna com o URI da faixa sem 'spotify:track:'
new = df_library["uri"].str.split(":", expand = True)
df_library['track_uri'] = new[2]

df_library.head()

### Crie nosso dataframe final, df_tableau:

In [ ]:
# criar dict final como uma cópia df_stream
df_tableau = df_stream.copy()

# adicionar coluna verificando se a música transmitida está na biblioteca
# não usado neste projeto, mas pode ser útil para visualizações legais
df_tableau['In Library'] = np.where(df_tableau['UniqueID'].isin(df_library['UniqueID'].tolist()),1,0)

# left join com df_library em UniqueID para trazer album e track_uri
df_tableau = pd.merge(df_tableau, df_library[['album','UniqueID','track_uri']],how='left',on=['UniqueID'])

df_tableau.head()

### Etapa 3: criar um novo projeto do Spotify <br>
### Faça login na sua conta de desenvolvedor aqui . No seu painel, crie um novo projeto. Depois de criado, você pode recuperar seu 'ID do cliente' e 'Segredo do cliente'. 
> https://developer.spotify.com/

### Etapa 4: criar um dataframe de gênero usando a API do Spotify <br>
### Primeiro, usaremos nosso Client ID e Client Secret para gerar um token de acesso para que possamos extrair dados da API do Spotify. Nota: este token deve ser regenerado após uma hora. 
> https://stmorse.github.io/journal/spotify-api.html

In [ ]:
# Criando Dataframe de Gênero

# salve seus IDs do novo projeto no Spotify Developer Dashboard
CLIENT_ID = 'your-client-id'
CLIENT_SECRET = 'your-client-secret'

In [ ]:
# Gerar token de acesso

# URL de autenticação
AUTH_URL = 'https://accounts.spotify.com/api/token'

# POST
auth_response = requests.post(AUTH_URL, {
    'grant_type': 'client_credentials',
    'client_id': CLIENT_ID,
    'client_secret': CLIENT_SECRET,
})

# converter a resposta para JSON
auth_response_data = auth_response.json()

# salve o token de acesso
access_token = auth_response_data['access_token']

In [ ]:
# usado para autenticar todas as chamadas de API
headers = {'Authorization': 'Bearer {token}'.format(token=access_token)}

In [ ]:
# URL base de todos os endpoints da API do Spotify
BASE_URL = 'https://api.spotify.com/v1/'

### Agora, vamos extrair o artista e os gêneros associados a cada track_uri em nossa biblioteca e adicioná-los a um dicionário (confira o console do Spotify para descobrir como extrair os pontos de dados de seu interesse).
> https://developer.spotify.com/documentation/web-api

In [ ]:
# crie um dicionário em branco para armazenar o URI da faixa, o URI do artista e os gêneros
dict_genre = {}

# converter a coluna track_uri em uma lista iterável
track_uris = df_library['track_uri'].to_list()

# percorrer os URIs da faixa e extrair o URI do artista usando a API,
# em seguida, use o URI do artista para extrair os gêneros associados a esse artista
# armazene tudo isso em um dicionário
for t_uri in track_uris:
    
    dict_genre[t_uri] = {'artist_uri': "", "genres":[]}
    
    r = requests.get(BASE_URL + 'tracks/' + t_uri, headers=headers)
    r = r.json()
    a_uri = r['artists'][0]['uri'].split(':')[2]
    dict_genre[t_uri]['artist_uri'] = a_uri
    
    s = requests.get(BASE_URL + 'artists/' + a_uri, headers=headers)
    s = s.json()
    dict_genre[t_uri]['genres'] = s['genres']

### Vamos converter este dicionário para um dataframe ( df_genre ) e expandi-lo para que cada gênero de cada faixa/artista esteja em sua própria linha (usei esta solução). Isso criará df_genre_expanded .

In [ ]:
# converter dicionário em dataframe com track_uri como a primeira coluna
df_genre = pd.DataFrame.from_dict(dict_genre, orient='index')
df_genre.insert(0, 'track_uri', df_genre.index)
df_genre.reset_index(inplace=True, drop=True)

df_genre.head()

In [ ]:
df_genre_expanded = df_genre.explode('genres')
df_genre_expanded.head()

### Em seguida, salvaremos df_tableau e df_genre_expanded como arquivos csv que podemos carregar no Tableau.

In [ ]:
# salve df_tableau e df_genre_expanded como arquivos csv que podemos carregar no Tableau
df_tableau.to_csv('MySpotifyDataTable.csv')
df_genre_expanded.to_csv('GenresExpandedTable.csv')

print('done')

### Etapa 5: Carregando dados no Tableau <br>
### Conecte-se ao seu arquivo Excel (MySpotifyDataTable.csv) como uma fonte de dados. Isso deve abrir seu arquivo GenresExpandedTable.csv no lado esquerdo também. Arraste o último arquivo para o lado direito e adicione um relacionamento entre as duas tabelas. Certifique-se de criar o relacionamento com base em track_uri (observação: talvez seja necessário clicar na seta para baixo ao lado de GenresExpandedTable.csv e clicar em "Field names are in first row") .

### Etapa 6: Editar campos no Tableau <br>
### Adicionaremos dois campos calculados à nossa tabela de dados. Poderíamos ter feito isso usando Python, mas eu queria experimentar o Tableau.

    Converta Ms Tocadas em Minutos Ouvidos :
    [Ms Tocadas]/60000
    
    Converta a hora final (UTC) para a hora final (Adj) (seu fuso horário, considerando o horário de verão):
    IF [Hora final] < DATE(2020–03–08) THEN DATEADD('hour',-5,[ Hora de término])
    ELSEIF [Hora de término] < DATE(2020–11–01) THEN DATEADD('hora',-4,[Hora de término])
    ELSE DATEADD('hora',-5,[Hora de término])
    END

### Etapa 7: criar visualizações <br>
### Baixe o painel para ver como criei as visualizações abaixo. Observe que eles estão todos vinculados no painel e, portanto, podem filtrar uns aos outros, o que é muito ruim!

> https://public.tableau.com/app/profile/anne.bode/viz/SpotifyPREMIUMDashboard/PremiumDashboard

1 - Principais artistas : os artistas mais populares por minutos ouvidos, com músicas individuais formando segmentos coloridos. <br>
2 - Amplitude do artista : número de músicas de cada artista que ouvi por > 1 minuto e > 1 stream <br>
3 - Top Songs : músicas mais populares por minutos ouvidos <br>
4 - Gêneros : todos os gêneros que ouvi, com tamanho e cor indicando a contagem de streams <br>
5 - Ouvidoria por mês : autoexplicativo (editar 29/09: converti o campo “Na biblioteca” em uma dimensão de uma medida, para que eu pudesse incluí-lo em um gráfico de barras empilhadas para ver quantos minutos totais eu ouvi a cada mês , e quantos estavam na minha biblioteca) <br>
6 - Audição por hora : autoexplicativo <br>

### Para concluir <br>
### Há uma tonelada de informações interessantes que você pode obter ao analisar seus dados de streaming e biblioteca do Spotify - especialmente quando combinados com campos de dados adicionais obtidos por meio da API do Spotify (ou seja, gênero). Eu aprendi que realmente gostei de Julien Baker este ano e meu amor por Lorde não tem limite de tempo. Eu ouvi demais ' Gone ' de Adrianne Lenker em agosto (#sadgirlvibes), mas em dezembro minha música favorita foi, claro, ' All I Want For Christmas Is You' de Mariah Carey'. Agora que sei como me conectar à API do Spotify, em projetos futuros quero verificar as qualidades musicais das músicas/artistas que ouço (além do gênero). Um campo em que estou especialmente interessado é a “dançabilidade”, porque sempre que tenho a tarefa de discotecar uma viagem em família, o único critério de minha mãe é “algo que eu possa dançar”. O que quer que isso signifique.